# Libraries Used

In [ ]:
import numpy as np
import pandas as pd
import random as r
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Dense, Conv1D, Flatten, AveragePooling1D, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# Data Preprocessing

In [ ]:
def preprocess_data(N, epsilon, S):
    data = []
    labels = []

    csv_data = [pd.read_csv(f"/Users/Jerry/Downloads/Caltech Research Datasets/FullDataset/W{i+1}.csv").values for i in range(5)]

    for _ in range(S):
        file_idx = r.randint(0, 4)
        file_data = csv_data[file_idx]

        if N > len(file_data):
            raise Exception("N is greater than the length of the file data.")

        start_idx = r.randint(0, len(file_data) - N)

        section = file_data[start_idx:start_idx + N]

        noise = np.random.normal(0, epsilon, section.shape)
        section = section + noise

        data.append(section)
        labels.append(file_idx + 1)

    data = np.array(data)
    labels = np.array(labels)

    num_classes = 5
    labels_one_hot = np.eye(num_classes)[labels - 1]

    return data, labels_one_hot

sections, labels_one_hot = preprocess_data(2000, 0.01, 200)

# CNN Model Training Zone

In [ ]:
input_shape = (sections.shape[1], sections.shape[2])
num_classes = 5
# start changing network
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='gelu', input_shape=input_shape),
    AveragePooling1D(pool_size=2),
    Flatten(),
    Dropout(0.7),
    Dense(100, activation='softmax'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

X_train, X_val, y_train, y_val = train_test_split(sections, labels_one_hot, test_size=0.2, random_state=42)

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Fit the model with early stopping
history = model.fit(X_train, y_train,
                    epochs=100,  # Increased the number of epochs
                    batch_size=16,  # Adjusted batch size
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping],
                    verbose=2)

model.summary(), history.history

Epoch 1/100
10/10 - 1s - loss: 1.6187 - accuracy: 0.1937 - val_loss: 1.5918 - val_accuracy: 0.5250 - 1s/epoch - 138ms/step
Epoch 2/100
10/10 - 1s - loss: 1.5736 - accuracy: 0.3187 - val_loss: 1.5296 - val_accuracy: 0.3500 - 705ms/epoch - 70ms/step
Epoch 3/100
10/10 - 1s - loss: 1.5085 - accuracy: 0.3938 - val_loss: 1.4966 - val_accuracy: 0.2750 - 688ms/epoch - 69ms/step
Epoch 4/100
10/10 - 1s - loss: 1.4827 - accuracy: 0.3688 - val_loss: 1.4777 - val_accuracy: 0.3500 - 689ms/epoch - 69ms/step
Epoch 5/100
10/10 - 1s - loss: 1.4675 - accuracy: 0.3938 - val_loss: 1.4562 - val_accuracy: 0.5000 - 774ms/epoch - 77ms/step
Epoch 6/100
10/10 - 1s - loss: 1.4436 - accuracy: 0.4875 - val_loss: 1.4260 - val_accuracy: 0.4750 - 831ms/epoch - 83ms/step
Epoch 7/100
10/10 - 1s - loss: 1.4161 - accuracy: 0.5938 - val_loss: 1.3954 - val_accuracy: 0.5000 - 841ms/epoch - 84ms/step
Epoch 8/100
10/10 - 1s - loss: 1.3944 - accuracy: 0.5250 - val_loss: 1.3768 - val_accuracy: 0.5500 - 833ms/epoch - 83ms/step
Ep

(None,
 {'loss': [1.6187198162078857,
   1.5735626220703125,
   1.5085471868515015,
   1.4826533794403076,
   1.467527985572815,
   1.4436274766921997,
   1.416090726852417,
   1.3944379091262817,
   1.3741819858551025,
   1.358614206314087,
   1.3403493165969849,
   1.32387375831604,
   1.3082150220870972,
   1.2942074537277222,
   1.2805984020233154,
   1.2677475214004517,
   1.2550262212753296,
   1.2429404258728027,
   1.2309799194335938,
   1.2191592454910278,
   1.2077691555023193,
   1.1965116262435913,
   1.1856579780578613,
   1.174760103225708,
   1.164224624633789,
   1.1537551879882812,
   1.143431305885315,
   1.1334999799728394,
   1.1235425472259521,
   1.1136882305145264,
   1.1042567491531372,
   1.0947452783584595,
   1.0852348804473877,
   1.0761325359344482,
   1.0670894384384155,
   1.0579875707626343,
   1.0491329431533813,
   1.0404807329177856,
   1.0319738388061523,
   1.0233523845672607,
   1.0149978399276733,
   1.0066204071044922,
   0.9986039996147156,
   0

## Plotting results

In [ ]:
# Extract loss and accuracy from the history object
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(1, len(train_loss) + 1)  # Assuming epochs index from 1

# Plotting training and validation loss
plt.figure(figsize=(12, 5))

# Loss Plot
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy Plot
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
plt.plot(epochs, train_accuracy, label='Training Accuracy')
plt.plot(epochs, val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## Saving the model

In [ ]:
model.save("/Users/Jerry/Downloads/Caltech Research Datasets/CNN_.h5")

C:\Users\Jerry\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Importing the Pre-trained CNN model

In [ ]:
new_model = model

In [ ]:
new_model = tf.keras.models.load_model('/Users/Jerry/Downloads/Caltech Research Datasets/CNN.h5')

# Testing Jupiter and Earth Classifications

In [ ]:
def preprocess_test_data():
    E_443 = pd.read_csv('/Users/Jerry/Downloads/Caltech Research Datasets/ContinuousDataset/Earth_443.csv').values
    E_551 = pd.read_csv('/Users/Jerry/Downloads/Caltech Research Datasets/ContinuousDataset/Earth_551.csv').values
    E_764 = pd.read_csv('/Users/Jerry/Downloads/Caltech Research Datasets/ContinuousDataset/Earth_764.csv').values

    J_443 = pd.read_csv('/Users/Jerry/Downloads/Caltech Research Datasets/ContinuousDataset/Jupiter_443.csv').values
    J_551 = pd.read_csv('/Users/Jerry/Downloads/Caltech Research Datasets/ContinuousDataset/Jupiter_551.csv').values
    J_764 = pd.read_csv('/Users/Jerry/Downloads/Caltech Research Datasets/ContinuousDataset/Jupiter_764.csv').values

    Earth_data = np.concatenate((E_443, E_551, E_764), axis=1)
    Jupiter_data = np.concatenate((J_443, J_551, J_764), axis=1)

    return Earth_data, Jupiter_data

Earth_data, Jupiter_data = preprocess_test_data()

In [ ]:
earth = np.pad(Earth_data, ((0, 0), (0, 2)), mode='constant')
jupiter = np.pad(Jupiter_data, ((0, 0), (0, 2)), mode='constant')

In [ ]:
padded_data = np.pad(earth, ((0, 1637), (0, 0)), mode='constant')
reshaped_data = padded_data.reshape((1, padded_data.shape[0], padded_data.shape[1]))
predictionsEarth = new_model.predict(reshaped_data)
print(predictionsEarth)

1/1 [==============================] - 0s 22ms/step
[[1.0885820e-03 4.7938074e-03 4.5523360e-02 8.8090834e-05 9.4850612e-01]]


In [ ]:
padded_data = np.pad(jupiter, ((0, 1637), (0, 0)), mode='constant')
reshaped_data = padded_data.reshape((1, padded_data.shape[0], padded_data.shape[1]))
predictionsJupiter = new_model.predict(reshaped_data)
print(predictionsJupiter)

1/1 [==============================] - 0s 15ms/step
[[1.4835221e-02 1.0523765e-03 9.8349643e-01 6.0177456e-07 6.1538321e-04]]


# Classified Results for Earth and Jupiter

In [ ]:
earth_predicted_classes = np.argmax(predictionsEarth, axis=1)
jupiter_predicted_classes = np.argmax(predictionsJupiter, axis=1)

print("Predicted complexities for Earth dataset:", earth_predicted_classes + 1)
print("Predicted complexities for Jupiter dataset:", jupiter_predicted_classes + 1)

Predicted complexities for Earth dataset: [5]
Predicted complexities for Jupiter dataset: [3]
